# Introduction - Oracle Version 2

Notebook to generate the Feat_BR oracles from the taskruns of the experts app (version 1 and 2) and volunteers app and compare the answers after the **shifting** of them.

The purpose of this notebook is to analyse the answers and the PyBossa applications **bug** found.

# Load Libraries and Data

In [1]:
from mod_finder_util import mod_finder_util
mod_finder_util.add_modules_origin_search_path()

import pandas as pd
import numpy as np

from modules.utils import aux_functions
from modules.utils import data_origin as do
from modules.utils import firefox_dataset_p2 as fd
from modules.utils import br_feat_orc_creator as orc_creator

from sklearn.metrics import cohen_kappa_score

bugreports = fd.Datasets.read_selected_bugreports_df()
features = fd.Datasets.read_features_df()

SelectedBugReports.shape: (91, 18)
Features.shape: (19, 8)


# Load Datasets

In [2]:
taskruns_expert = fd.TaskRuns.read_expert_taskruns_df()
taskruns_expert_2 = fd.TaskRuns.read_expert_taskruns_2_df()

taskruns_volunteers_1 = fd.TaskRuns.read_volunteers_taskruns_1_df()
taskruns_volunteers_2 = fd.TaskRuns.read_volunteers_taskruns_2_df()

TaskRuns shape: (93, 11)
TaskRuns shape: (93, 11)
TaskRuns_1 shape: (113, 11)
TaskRuns_2 shape: (10, 11)


# Create BR_2_Feature Matrices

In [3]:
creator = orc_creator.Br_Feat_Oracle_Creator(bugreports, features)
creator.create_br_feat_expert_matrix(taskruns_expert)
creator.create_br_feat_expert_2_matrix(taskruns_expert_2)
creator.create_br_feat_volunteers_matrix(taskruns_volunteers_1, taskruns_volunteers_2)

Feat_BR Expert Matrix shape: (91, 19)
Feat_BR Expert 2 Matrix shape: (91, 19)
Feat_BR Volunteers Matrix shape: (91, 19)


# Load BR_2_Features Matrices

In [4]:
expert_matrix = fd.Feat_BR_Oracles.read_feat_br_expert_df()
expert_matrix_2 = fd.Feat_BR_Oracles.read_feat_br_expert_2_df()
volunteers_matrix = fd.Feat_BR_Oracles.read_feat_br_volunteers_df()

Feat_BR Expert Matrix shape: (91, 19)
Feat_BR Expert 2 Matrix shape: (91, 19)
Feat_BR Volunteers Matrix shape: (91, 19)


# Cohen Kappa Expert Matrix x Expert 2 Matrix

In [5]:
expert_answers = []
expert_2_answers = []

for idx,row in expert_matrix_2.iterrows():
    for col in expert_matrix_2.columns:
        expert_answers.append(expert_matrix.at[idx,col])
        expert_2_answers.append(expert_matrix_2.at[idx,col])

print("Expert Answers Length: {}".format(len(expert_answers)))
print("Expert 2 Answers Length: {}".format(len(expert_2_answers)))

print("Cohen Kappa Score: {}".format(cohen_kappa_score(expert_answers, expert_2_answers)))

Expert Answers Length: 1729
Expert 2 Answers Length: 1729
Cohen Kappa Score: 0.8755237505579474


# Get Features by Bug Reports

In [7]:
def get_features(br_id, data_origin):
    features_ids = ""
    matrix = ""
    
    if data_origin == do.DataOrigin.EXPERT:
        matrix = expert_matrix
    elif data_origin == do.DataOrigin.EXPERT_2:
        matrix = expert_matrix_2
    elif data_origin == do.DataOrigin.VOLUNTEERS:
        matrix = volunteers_matrix
    
    for col in matrix.columns:
        if matrix.at[br_id, col] == 1:
            if features_ids == "":
                features_ids = str(matrix.columns.get_loc(col) + 1)
            else:
                features_ids = features_ids + " " + str(matrix.columns.get_loc(col) + 1)
            
    return features_ids

# Get Trace Links between Bug Reports and Features 

In [8]:
br_2_feat_m_final = pd.DataFrame()
br_2_feat_m_final['Bug_Number'] = bugreports.Bug_Number
br_2_feat_m_final['fids_exp'] = bugreports.apply(lambda row : get_features(row['Bug_Number'], do.DataOrigin.EXPERT), axis=1)
br_2_feat_m_final['fids_exp_2'] = bugreports.apply(lambda row : get_features(row['Bug_Number'], do.DataOrigin.EXPERT_2), axis=1)
br_2_feat_m_final['fids_vol'] = bugreports.apply(lambda row : get_features(row['Bug_Number'], do.DataOrigin.VOLUNTEERS), axis=1)

br_2_feat_m_final.replace(" ", "", inplace=True)
br_2_feat_m_final.set_index('Bug_Number', inplace=True)

br_2_feat_m_final['f_names_exp'] = br_2_feat_m_final.fids_exp.apply(lambda row : [features[features.Feature_Number == int(f_id)].Feature_Shortname for f_id in row.split(" ") if f_id != ""])
br_2_feat_m_final['f_names_exp_2'] = br_2_feat_m_final.fids_exp_2.apply(lambda row : [features[features.Feature_Number == int(f_id)].Feature_Shortname for f_id in row.split(" ") if f_id != ""])
br_2_feat_m_final['f_names_vol'] = br_2_feat_m_final.fids_vol.apply(lambda row : [features[features.Feature_Number == int(f_id)].Feature_Shortname for f_id in row.split(" ") if f_id != ""])
br_2_feat_m_final['bug_title'] = br_2_feat_m_final.apply(lambda row : bugreports[bugreports.Bug_Number == int(row.name)].Summary.values[0], axis=1)

display(br_2_feat_m_final.iloc[0:30,:])
display(br_2_feat_m_final.iloc[30:60,:])
display(br_2_feat_m_final.iloc[60:99,:])

,fids_exp,fids_exp_2,fids_vol,f_names_exp,f_names_exp_2,f_names_vol,bug_title
Bug_Number,,,,,,,
1248267,6,6,6,[[context_menu]],[[context_menu]],[[context_menu]],"Right click on bookmark item of ""Recently Book..."
1248268,,,4,[],[],[[browser_customization]],"Unable to disable ""Recently bookmarked"""
1257087,,,,[],[],[],Middle mouse click on history item would not open
1264988,1,1,1,[[new_awesome_bar]],[[new_awesome_bar]],[[new_awesome_bar]],Scrollbar appears for a moment in the new Awes...
1267480,,,,[],[],[],Intermittent browser/components/preferences/in...
1267501,3,3,,[[apz_async_scrolling]],[[apz_async_scrolling]],[],New Private Browsing start-page overflows off ...
1269348,,,,[],[],[],Show last sync date tooltip on Synced Tabs sid...
1269485,3,3,,[[apz_async_scrolling]],[[apz_async_scrolling]],[],New Private Browsing start-page has white/gray...
1270274,,,,[],[],[],Intermittent browser_share.jsuncaught exceptio...


,fids_exp,fids_exp_2,fids_vol,f_names_exp,f_names_exp_2,f_names_vol,bug_title
Bug_Number,,,,,,,
1291770,,,1,[],[],[[new_awesome_bar]],"[HiDPI] Globe icon is too small, ends up being..."
1292566,15,15,15,[[downloads_dropmaker]],[[downloads_dropmaker]],[[downloads_dropmaker]],"The ""open"" button in the subview for temporari..."
1292573,15,15,15,[[downloads_dropmaker]],[[downloads_dropmaker]],[[downloads_dropmaker]],Broken keyboard interaction with the subview i...
1293308,1,1,1,[[new_awesome_bar]],[[new_awesome_bar]],[[new_awesome_bar]],System font setting respected by location bar ...
1294733,1,1,1,[[new_awesome_bar]],[[new_awesome_bar]],[[new_awesome_bar]],Awesomebar search buttons show auto completed ...
1294887,1,1,1,[[new_awesome_bar]],[[new_awesome_bar]],[[new_awesome_bar]],Middle clicking search results does not open a...
1295502,,,3 10 16 17 19,[],[],"[[apz_async_scrolling], [webgl_comp], [webgl2]...","2.1 - 4.09% tart / tresize / tsvgx (linux64,..."
1296322,18,18,18,[[indicator_device_perm]],[[indicator_device_perm]],[[indicator_device_perm]],X button is vertically misaligned on the permi...
1296366,1,1,1,[[new_awesome_bar]],[[new_awesome_bar]],[[new_awesome_bar]],"Ctrl+Click awesomebar entry with ""Switch to Ta..."


,fids_exp,fids_exp_2,fids_vol,f_names_exp,f_names_exp_2,f_names_vol,bug_title
Bug_Number,,,,,,,
1313969,1,1,6,[[new_awesome_bar]],[[new_awesome_bar]],[[context_menu]],"[Follow-up] Ctrl+Click awesomebar entry with ""..."
1314643,1,1 6,9 12 15,[[new_awesome_bar]],"[[new_awesome_bar], [context_menu]]","[[tts_in_rm], [pointer_lock_api], [downloads_d...",[One-off searches] Changing the default search...
1316126,,,14 17,[],[],"[[zoom_indicator], [flac_support]]",`_endRemoveTab` can get stuck in an infinite loop
1318903,4,4,4,[[browser_customization]],[[browser_customization]],[[browser_customization]],[Windows 7 and below] Fullscreen window contro...
1319119,,,1,[],[],[[new_awesome_bar]],Turn on Insecure Password Warning in Firefox R...
1319433,,,,[],[],[],Devices differ in Synced Tab and Send to Devic...
1319919,,,,[],[],[],Refine Insecure Password Warning style
1320557,,,,[],[],[],No security information displayed for a specif...
1323211,,,,[],[],[],Change Sync datatype label from Tabs to Open tabs
